### LangChain Reasoning and Synthesis Demonstration
This notebook demonstrates the use of LangChain to perform reasoning tasks using local models via Ollama.
We will break down a complex question into logical steps, retrieve relevant information for each step, and synthesize a final answer.

In [1]:
# !pip install wikipedia

In [2]:
# Imports
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap
import re

### Load Local Reasoning and Synthesis Models
Here, we initialize two models using Ollama:
- **Reasoning Model (`phi4-mini`)**: Used to break down questions into logical steps.
- **Synthesis Model (`Gemma3:1b`)**: Used to synthesize a final answer based on retrieved information.
The `try` block ensures that the models are loaded successfully, and if there is an error (e.g., Ollama is not running), it provides helpful debugging information.

In [3]:
try:
    reasoning_llm = OllamaLLM(model="phi4-mini")
    synthesis_llm = OllamaLLM(model="Gemma3:1b")
except Exception as e:
    print("❌ Failed to connect to Ollama or load model 'phi4-mini'.")
    print("💡 Make sure Ollama is running and the model is available:")
    print("    ollama run phi4-mini")
    print(f"Error details: {e}")
    exit(1)

### Define Reasoning Prompt
We define a prompt template to guide the reasoning model. The template instructs the model to break down a given question into logical steps.
The `PromptTemplate.from_template` method allows us to create a reusable template with placeholders (e.g., `{question}`) that can be dynamically filled with input data.

In [4]:
reasoning_prompt = PromptTemplate.from_template("""
You are a reasoning assistant. Break the following question into logical steps. Use the provided information to answer the question:

Question: {question}

Step-by-step breakdown:
""")

### Chain Prompt and Model
We use LangChain's Expression Language to chain the reasoning prompt with the reasoning model. This creates a pipeline where the input question is first processed by the prompt and then passed to the reasoning model for step-by-step breakdown.

In [5]:
step_chain = reasoning_prompt | reasoning_llm

### Invoke Reasoning Chain
We provide a question to the reasoning chain and invoke it to generate a step-by-step breakdown of the question.
The `invoke` method processes the input through the chain and returns the model's response.

In [6]:
question = "Who was the U.S. president during the moon landing, and what was his policy on space exploration?"
response = step_chain.invoke({"question": question})
print("💡 Reasoning steps from the model:\n")
print(response)

💡 Reasoning steps from the model:

To address this query logically:


1. Identify when NASA's Apollo 11 mission landed people safely back onto Earth.

2. Determine who served as President of the United States at that time.


Apollo 11 successfully completed its historic task in July 1969, landing astronauts Neil Armstrong and Buzz Aldrin on lunar soil while Michael Collins orbited above them aboard their spacecraft Columbia (ASTM reference: Apollo mission timeline).


3. Find out which U.S. president was serving during this period.

The President of the United States at that time had to have been Richard Nixon.


4. Research Mr. Nixon's policies related specifically to space exploration, as those were his beliefs and directives concerning NASA’s initiatives in outer-space projects up until he left office on August 9, 1974 (using sources such as historical archives or official government websites).


Upon leaving the White House after losing re-election against Jimmy Carter during Novem

### Call the Wikidpedia API
This code uses LangChain tools to interact with Wikipedia. It breaks a question into reasoning steps, retrieves facts from Wikipedia for each step, and displays the results.

---

### Key Steps

1. **Initialize Tools**: Set up `WikipediaAPIWrapper` and `WikipediaQueryRun` for Wikipedia lookups.
2. **Process Question**: Use `step_chain.invoke()` to break the question into steps.
3. **Retrieve Facts**: Look up each step using the Wikipedia tool, handle errors, and collect results.
4. **Display Results**: Combine retrieved facts into a single output and print them.

---

Let me know if you'd like further refinements! 🚀

In [7]:
# --- Correct setup for Wikipedia Tool ---
from langchain.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

wiki_api = WikipediaAPIWrapper(top_k_results=1, lang="en")
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api)

# --- Use Wikipedia for each reasoning step ---

steps_text = step_chain.invoke({"question": question})

print("\n🧠 Reasoning steps:\n")
print(steps_text)

# Extract lines that look like steps
step_lines = re.findall(r"\d+\.\s+(.*)", steps_text)
facts = []

for step in step_lines:
    print(f"🔍 Looking up: {step}")
    try:
        result = wiki_tool.run(step)
    except Exception as e:
        result = f"Error retrieving from Wikipedia: {e}"
    facts.append(f"- {step.strip()}: {result}")

combined_facts = "\n".join(facts)

print("\n📚 Retrieved facts from Wikipedia:\n")
print(combined_facts)



🧠 Reasoning steps:

Sure! Let's break down this problem step by step using some known historical facts.

1. Identify when humans first landed on the Moon.
   - Humans successfully walked on the surface of the Moon for NASA's Apollo 11 mission in July 1969, with astronauts Neil Armstrong and Buzz Aldrin walking while Michael Collins orbited above them aboard Lunar Module Eagle.

2. Determine who was president at that time (July 1969).
   - In July 1969, Richard Nixon served as President of the United States because he succeeded Lyndon B. Johnson on August 22, 1968 and took office nine months later; thus, by mid-September to late-July would be his presidency.

3. Investigate any existing policies or statements regarding space exploration from that period.
   - Richard Nixon's administration (1969–1974) was known for continuing the commitment initiated under John F. Kennedy’s Presidency during NASA Apollo missions in 1961, which focused on landing a man safely back to Earth by December 2

### Parse and Retrieve Information
We extract the steps from the reasoning response using a regular expression (`re.findall`) to match numbered steps.
For each step, we use the `fake_lookup` function to retrieve relevant information and store the results in a list (`facts`).
Finally, the facts are combined into a single string (`combined_facts`) for further processing.

### Define Synthesis Prompt
We define a prompt template to guide the synthesis model. The template combines the question and retrieved facts to generate a complete answer.
The `PromptTemplate.from_template` method is used to create a structured template with placeholders for the question and facts.

In [8]:
synthesis_prompt = PromptTemplate.from_template("""
Based on the following question and information, write a complete answer.

Question: {question}

Information:
{facts}

Answer:
""")
synthesis_chain = synthesis_prompt | synthesis_llm

### Generate Final Answer
We invoke the synthesis chain to generate the final answer based on the question and the retrieved facts.
The `invoke` method processes the input through the synthesis chain and returns the model's response, which is the final synthesized answer.

In [9]:
final_answer = synthesis_chain.invoke({
    "question": question,
    "facts": combined_facts
})
print("\n🧠 Final answer from synthesis model:\n")
print(final_answer)


🧠 Final answer from synthesis model:

Here’s a comprehensive answer based on the provided information, addressing the question of who the U.S. president was during the moon landing and their policy on space exploration:

**Who was the U.S. president during the moon landing?**

The U.S. president during the Apollo 11 moon landing was **John F. Kennedy**.

**What was his policy on space exploration?**

Kennedy's policy on space exploration was centered around a bold goal: to land a man on the Moon and return him safely to Earth before the end of the decade. This was a crucial part of a broader strategy to demonstrate U.S. technological superiority and secure America’s place in the “space race” against the Soviet Union.  His justification for this ambitious goal was framed within the context of the Cold War, highlighting the strategic importance of achieving a victory in this competition.

**Investigate any existing policies or statements regarding space exploration from that period:**

